In [1]:
# GENERATE NEW CUIS INSTEAD OF MESH TERMS

# This code will redo the concept extraction of the full database

In [2]:
from trialstreamer import dbutil
import psycopg2
import tqdm
import json
from robotreviewer.textprocessing import minimap

In [ ]:
# first transfer pre-done

import pandas
import json
import tqdm

pmids_done = set()

df.read_csv('~/ijmarshall/cuis.csv')

for i, r in tqdm.tqdm(df.iterrows()):
    cur.execute('update pubmed_annotations set population_mesh=(%s), interventions_mesh=(%s), outcomes_mesh=(%s) where pmid=(%s);',
                (json.dumps(r['population_mesh']), json.dumps(r['interventions_mesh']), json.dumps(r['outcomes_mesh']), r['pmid']))
    pmids_done.add(r['pmid'])

In [22]:
# get the pico span data

cur = dbutil.db.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
cur.execute('select pmid, population, interventions, outcomes from pubmed_annotations;')
results_in = [dict(r) for r in cur if r['pmid'] not in pmids_done]

In [23]:
# this should fix up PubMed
fields = ['population', 'interventions', 'outcomes']

cur = dbutil.db.cursor()

for in_r in tqdm.tqdm(results_in):
    out_r = {}
    for f in fields:
        out_r[f] = minimap.get_unique_terms(in_r[f])
    cur.execute('update pubmed_annotations set population_mesh=(%s), interventions_mesh=(%s), outcomes_mesh=(%s) where pmid =(%s)',
                   (json.dumps(out_r['population']), json.dumps(out_r['interventions']), json.dumps(out_r['outcomes']), in_r['pmid']))


100%|██████████| 44/44 [00:01<00:00, 32.24it/s]


In [12]:
dbutil.db.commit()

In [14]:
# get the pico span data

cur = dbutil.db.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
cur.execute('select id, population, interventions, outcomes from medrxiv_covid19;')
results_in = [dict(r) for r in cur]

In [15]:
# this should fix up PubMed
fields = ['population', 'interventions', 'outcomes']

cur = dbutil.db.cursor()

for in_r in tqdm.tqdm(results_in):
    out_r = {}
    for f in fields:
        out_r[f] = minimap.get_unique_terms(in_r[f])
    cur.execute('update medrxiv_covid19 set population_mesh=(%s), interventions_mesh=(%s), outcomes_mesh=(%s) where id =(%s)',
                   (json.dumps(out_r['population']), json.dumps(out_r['interventions']), json.dumps(out_r['outcomes']), in_r['id']))

100%|██████████| 44/44 [00:01<00:00, 25.13it/s]


In [11]:
# get the pico span data

cur = dbutil.db.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
cur.execute('select id, population, interventions, outcomes from ictrp;')
results_in = [dict(r) for r in cur]

In [12]:
# this should fix up ICTRP
fields = ['population', 'interventions', 'outcomes']

cur = dbutil.db.cursor()


with tqdm.tqdm(total = len(results_in)) as pbar:

    while results_in:
        in_r = results_in.pop()
        out_r = {}
        for f in fields:
            out_r[f] = minimap.get_unique_terms(in_r[f])
        cur.execute('update ictrp set population_mesh=(%s), interventions_mesh=(%s), outcomes_mesh=(%s) where id =(%s)',
                       (json.dumps(out_r['population']), json.dumps(out_r['interventions']), json.dumps(out_r['outcomes']), in_r['id']))
        pbar.update()

100%|██████████| 241378/241378 [7:32:49<00:00,  8.88it/s]      


In [16]:
dbutil.db.commit()